# StatVid — Data Wrangling

## Summary

This notebook documents the reasoning process and design decisions of StatVid's YouTube API data pipeline. We'll look at how to interface with the API, understand the response schema, evaluate which fields are worth ingesting into our bronze data layer, and formulate the final structure of our pipeline.

*Note on reproducibility:*
This notebook does not contain our actual ingestion system. The source code for that can be found under the [ingest directory](../src/statvid/ingest/).
Additionally, this notebook pulls live data directly from the YouTube Data API v3. Thus, the data in this notebook is nondeterministic, subject to YouTube quota limits, and dependent on API versioning.

## Content

- [Exploring YouTube's API](#exploring-youtubes-api)
- [Defining Our Data](#defining-our-data)


<a id="exploring-youtubes-api"></a>
## Exploring YouTube's API

We'll start off by testing out YouTube's Data API v3. Documentation for how to use the API can be found here: [YouTube Data API v3](https://developers.google.com/youtube/v3)


### Dependencies

Before running this notebook, you'll need:
- Google's API Python client library: [GitHub repo](https://github.com/googleapis/google-api-python-client)
- A YouTube Data API v3 key: [Google Cloud Console](https://console.cloud.google.com/)

In [1]:
import os
from datetime import datetime, timezone

import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv
from googleapiclient.discovery import build

First we need to construct our `youtube` client object. This object will handle all of our data requests throughout the notebook.

In [2]:
load_dotenv()

API_KEY = os.getenv("YOUTUBE_API_KEY")
if not API_KEY:
    raise RuntimeError("YOUTUBE_API_KEY not set in .env")

In [3]:
youtube = build("youtube", "v3", developerKey=API_KEY)
youtube

### Channels

We'll begin our exploration process by taking a look at channel objects.

In [4]:
def get_channel_by_handle (handle):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        forHandle=handle
    )
    return request.execute()

In [5]:
test_channel = get_channel_by_handle("@MrBeast")
test_channel

{'kind': 'youtube#channelListResponse',
 'etag': 'S9Tyq3ZzpZSvKA9bF0pVtKF81Uw',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '-Plt4xK2qnpWaxsR34661-IGAkI',
   'id': 'UCX6OQ3DkcsbYNE6H8uQQuVA',
   'snippet': {'title': 'MrBeast',
    'description': "SUBSCRIBE FOR A COOKIE!\nNew MrBeast or MrBeast Gaming video every single Saturday at noon eastern time!\nAccomplishments:\n- Raised $20,000,000 To Plant 20,000,000 Trees\n- Removed 30,000,000 pounds of trash from the ocean\n- Helped 2,000 people walk again\n- Helped 1,000 blind people see\n- Helped 1,000 deaf people hear\n- Built wells in Africa\n- Built and gave away 100 houses\n- Adopted every dog in a shelter (twice)\n- Given millions to charity\n- Started my own snack company Feastables\n- Started my own software company Viewstats\n- Started Lunchly, a tasty, better-for-you lunch option\n- Gave away a private island (twice)\n- Gave away 1 million meals\n- I counted to 100k\n- Ra

Based on the shape of `test_channel` we know that `youtube.channels().list()`, and likely all client calls, return a dictionary with a sort of "header" containing *'kind'*, *'etag'*, *'pageInfo'*. The actual data we want will usually be in an ***'items'*** list.

In [6]:
test_channel_handles = ["@MrBeast", "@veritasium", "HealthyGamerGG", "@Vanillamacee", "@JeffNippard", "@mitocw", "@boilerroom"]
test_channels = {
    handle: get_channel_by_handle(handle).get('items')[0]
    for handle in test_channel_handles
}
test_channels["@MrBeast"]

{'kind': 'youtube#channel',
 'etag': '-Plt4xK2qnpWaxsR34661-IGAkI',
 'id': 'UCX6OQ3DkcsbYNE6H8uQQuVA',
 'snippet': {'title': 'MrBeast',
  'description': "SUBSCRIBE FOR A COOKIE!\nNew MrBeast or MrBeast Gaming video every single Saturday at noon eastern time!\nAccomplishments:\n- Raised $20,000,000 To Plant 20,000,000 Trees\n- Removed 30,000,000 pounds of trash from the ocean\n- Helped 2,000 people walk again\n- Helped 1,000 blind people see\n- Helped 1,000 deaf people hear\n- Built wells in Africa\n- Built and gave away 100 houses\n- Adopted every dog in a shelter (twice)\n- Given millions to charity\n- Started my own snack company Feastables\n- Started my own software company Viewstats\n- Started Lunchly, a tasty, better-for-you lunch option\n- Gave away a private island (twice)\n- Gave away 1 million meals\n- I counted to 100k\n- Ran a marathon in the world's largest shoes\n- Survived 50 hours in Antarctica\n- Recreated Squid Game in real life\n- Created the largest competition show 

In [7]:
for channel in test_channels.values():
    channel_id = channel.get('id')
    title = channel.get('snippet').get('title')
    country = channel.get('snippet').get('country')

    raw_published = channel.get('snippet').get('publishedAt')
    published = datetime.fromisoformat(raw_published.replace("Z", "+00:00"))

    subcount = int(channel.get('statistics').get('subscriberCount'))
    vidcount = int(channel.get('statistics').get('videoCount'))
    
    print(
        f"{title}:\n"
        f"ID: {channel_id} | "
        f"Created: {published:%b %d, %Y} | "
        f"Subscribers: {subcount:,} | "
        f"Videos: {vidcount} | "
        f"Country: {country}\n\n"
    )

MrBeast:
ID: UCX6OQ3DkcsbYNE6H8uQQuVA | Created: Feb 20, 2012 | Subscribers: 454,000,000 | Videos: 929 | Country: US


Veritasium:
ID: UCHnyfMqiRRG1u-2MsSQLbXA | Created: Jul 21, 2010 | Subscribers: 19,700,000 | Videos: 473 | Country: US


HealthyGamerGG:
ID: UClHVl2N3jPEbkNJVx-ItQIQ | Created: Jan 17, 2019 | Subscribers: 3,150,000 | Videos: 2408 | Country: US


Vanillamace:
ID: UCIH0E-zQ3-HzYDChlpafRsQ | Created: Feb 06, 2024 | Subscribers: 1,460,000 | Videos: 84 | Country: None


Jeff Nippard:
ID: UC68TLK0mAEzUyHx5x5k-S1Q | Created: Apr 19, 2014 | Subscribers: 7,990,000 | Videos: 606 | Country: CA


MIT OpenCourseWare:
ID: UCEBb1b_L6zDS3xTUrIALZOw | Created: Oct 11, 2005 | Subscribers: 6,070,000 | Videos: 7838 | Country: US


Boiler Room:
ID: UCGBpxWJr9FNOcFYA5GkKrMg | Created: May 10, 2012 | Subscribers: 5,040,000 | Videos: 10183 | Country: GB




Unfortunately, the API only allows us to pull one channel at a time via handles. For our pipeline implementation we'll want to use the channel's *'id'* instead.

### Uploads

Next, let's try pulling a specific channel's uploads.

In [8]:
def get_playlist_items_by_id(playlist_id, max_results=5):
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        maxResults=max_results,
        playlistId=playlist_id
    )
    return request.execute()

In [9]:
test_playlist_id = test_channels["@MrBeast"].get('contentDetails').get('relatedPlaylists').get('uploads')
test_playlist = get_playlist_items_by_id(test_playlist_id, max_results=3).get('items')
test_playlist

[{'kind': 'youtube#playlistItem',
  'etag': 'vRMrXeSzSHRLI7Sms52WJtwICrU',
  'id': 'VVVYNk9RM0RrY3NiWU5FNkg4dVFRdVZBLmZPNUZ3QmNDSkJZ',
  'snippet': {'publishedAt': '2025-12-12T17:00:03Z',
   'channelId': 'UCX6OQ3DkcsbYNE6H8uQQuVA',
   'title': 'Would You Date Him for $10,000?',
   'description': '',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/sddefault.jpg',
     'width': 640,
     'height': 480},
    'maxres': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/maxresdefault.jpg',
     'width': 1280,
     'height': 720}},
   'channelTitle': 'MrBeast',
   'playlistId': 'UUX6OQ3DkcsbYNE6H8uQQuVA',
   'position': 0,
   'resourceId': {

Both *'snippet'* and *'contentDetails'* appear to contain our target, **'videoId'**. So we'll implement our PlaylistItems fetching method with just *'contentDetails'* later.

In [10]:
def get_videos_by_id(vid_ids: str):
    request = youtube.videos().list(
        part="snippet,contentDetails,statistics,topicDetails",
        id=vid_ids
    )
    return request.execute()

In [11]:
test_vid_ids = []
for video in test_playlist:
    test_vid_ids.append(video.get('contentDetails').get('videoId'))
test_ids = ','.join(test_vid_ids)
test_playlist_videos = get_videos_by_id(test_ids)
test_playlist_videos

{'kind': 'youtube#videoListResponse',
 'etag': 'wI9c5n4GqXxUEBMxc7iOzA1u3NM',
 'items': [{'kind': 'youtube#video',
   'etag': 'SKDqnq5YId7qm7qR0VwCyraH3Ss',
   'id': 'fO5FwBcCJBY',
   'snippet': {'publishedAt': '2025-12-12T17:00:03Z',
    'channelId': 'UCX6OQ3DkcsbYNE6H8uQQuVA',
    'title': 'Would You Date Him for $10,000?',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/fO5FwBcCJBY/maxresdefault.jpg',
      'width': 1280,
      'height': 720}},
    'channelTitle': 'MrBeast',
    'categoryId

A couple interesting findings to note:
- It appears that a channel's ID may be near identical to its uploads playlist ID. For the case of *@MrBeast*, his channel's ID `UCX6OQ3DkcsbYNE6H8uQQuVA` only differs from his upload playlist's ID `UUX6OQ3DkcsbYNE6H8uQQuVA` in a single letter. What's more, all channels from our 'test_channels' start with the letters "UC", likely standing for "user channel". This may mean that all upload playlists also start with "UU" (i.e. "user uploads").
  - While we could use this to our advantage to slightly improve our request overhead, it's probably a safer choice long term to continue pulling both resources for future maintainability.
- Documentation seems to indicate that Topic Details is, for the most part, a retired feature. As such, it seems likely that now it only outputs a Wikipedia entry of the video category.

We'll circle back to both of these theories in a bit, but for now we're going to shift gears into ironing out what exact data we're looking to pull.

<a id="defining-our-data"></a>
## Defining Our Data

The goal of StatVid is to create a model that can accurately generalize the average YouTube video for the sake of predicting viewership and engagement. However, it's important for us to refine our scope a bit, since predictions for the *true* average YouTube video would be of little value. This is because a vast majority of the content on YouTube is one-off uploads from individuals with no intent of building a subscriber base. If we were to generate a sample that was a true-to-form representation of YouTube's video population, over 90% of our sample would be blurry, minute-long videos of someone's cat.

So we want to create a model that generalizes YouTube content, but a specific kind of content. The content we're looking to represent is made by content creators, or "YouTubers". This is generally what one thinks of when they imagine a YouTube video. This type of content today is an even smaller chunk of YouTube's complete library as the platform continues to dominate the online video streaming space. There is now content produced from major production studios such as movies, TV-shows, music videos, news networks, etc. There's also "YouTube Shorts", YouTube's competitive response to the newest form of content creation in today's age, popularized by platforms such as TikTok. While predictive modeling in all of these avenues could be useful to some end users, these all lie far beyond the scope of StatVid, and will thus need to be filtered out of our initial data gathering process.

### Contstraints

To have our model effectively generalize the correct subset of content, we have to define the constraints that shape the data our model will fit to. 

**Content format:**
- Videos must be long-form. YouTube Shorts are excluded.

**Time window:**
- Videos must have been uploaded from within the last 2 years (Janurary 2024 - December 2025).
- Videos must be at least 1 month old.
- Views are measured in fixed early windows (7-day and 30-day views).

**Size:**
First, to ensure we have enough data to train our model, we'll be aiming to pull around 20,000 - 50,000 videos in total.

**Shape:**

Secondly, to ensure both even distribution and accurate representation, we'll have to be a bit smart about our sampling, specifically in regards to video **categories** and channels' **subscriber count**. We want a reasonably even 

- Data must be an evenly split distribution across YouTube video categories.
- Data should be logarthmically distributed across channel subscriber counts.
- YouTube channels must have uploaded at least 10 videos within the past 2 years.
- YouTube channels must have at least 10,000 subscribers.
- YouTube channels must be from the US (for simplicity sake).

### Categories
YouTube has a built in *Category* feature for video objects that defines generalized styles of content for the platform. While Categories are not an entirely accurate representation of the wide range of niche genres YouTube hosts, it can help us refine our model's subset even further.

In [12]:
# Fetch and display available YouTube video categories for the US region

request = youtube.videoCategories().list(
    part='snippet',
    regionCode="US"
)

response = request.execute()
categories = response.get('items', [])
categories[:5]

[{'kind': 'youtube#videoCategory',
  'etag': 'grPOPYEUUZN3ltuDUGEWlrTR90U',
  'id': '1',
  'snippet': {'title': 'Film & Animation',
   'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
 {'kind': 'youtube#videoCategory',
  'etag': 'Q0xgUf8BFM8rW3W0R9wNq809xyA',
  'id': '2',
  'snippet': {'title': 'Autos & Vehicles',
   'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
 {'kind': 'youtube#videoCategory',
  'etag': 'qnpwjh5QlWM5hrnZCvHisquztC4',
  'id': '10',
  'snippet': {'title': 'Music',
   'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
 {'kind': 'youtube#videoCategory',
  'etag': 'HyFIixS5BZaoBdkQdLzPdoXWipg',
  'id': '15',
  'snippet': {'title': 'Pets & Animals',
   'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
 {'kind': 'youtube#videoCategory',
  'etag': 'PNU8SwXhjsF90fmkilVohofOi4I',
  'id': '17',
  'snippet': {'title': 'Sports',
   'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}}]

In [13]:
categories = pd.json_normalize(categories)
categories['id'] = categories['id'].astype(int)
categories = (
    categories
    .drop(columns=['kind', 'etag', 'snippet.channelId'])
    .rename(columns={'snippet.title': "title", 'snippet.assignable': "assignable"})
    .set_index('id')
)
categories

,title,assignable
id,,
1,Film & Animation,True
2,Autos & Vehicles,True
10,Music,True
15,Pets & Animals,True
17,Sports,True
18,Short Movies,False
19,Travel & Events,True
20,Gaming,True
21,Videoblogging,False


The ***'assignable'*** feature plays an important role for us. This is YouTube's internal method of seperating regular, user-submitted, long-form video content from the rest of the content it hosts. YouTube in its current state features movies, TV shows, and trailers all uploaded by production companies. Additionally, YouTube also features "YouTube Shorts" which is their subplatform for short-form content, as a competitor of sorts to TikTok & Instagram Reels. Strangely, *'Videoblogging'* seems to stand out from these nonassignable categories for being the only one that represents a regular style of YouTube content. However, looking into this further it's likely that this category has been depreciated due to YouTube's shift of focus on content's topics over format.

Thus, we want our dataset to only include assignable categories.

In [14]:
# Creating new 'trainable' column to indicate if our model will use the category.
categories["trainable"] = categories['assignable']
categories[categories['trainable']]

,title,assignable,trainable
id,,,
1,Film & Animation,True,True
2,Autos & Vehicles,True,True
10,Music,True,True
15,Pets & Animals,True,True
17,Sports,True,True
19,Travel & Events,True,True
20,Gaming,True,True
22,People & Blogs,True,True
23,Comedy,True,True


Among this list, there are a few more categories that stand out as potentially too "different" from the average content on YouTube to include in our dataset. *"Music"*, *"News & Politics"*, and *"Nonprofits & Activism"* all include a large percentage of studio or organization produced content, and thus are likely to cause a distribution shift away from typical, content creator viewership. *"Film & Animation"* is an interesting one because most content in this category is driven by single content creators, however upload frequency and viewership patterns are dramatically different. Engagement tends to be consistently higher in this category, but at the cost of high production content which greatly affects the creator's upload schedule. This behavior of low uploads + high viewership is too unlike the content we would like our model to generalize, and so it will also be excluded from our dataset.

In [15]:
categories.loc[[1, 10, 25, 29], 'trainable'] = False
categories[categories['trainable']]

,title,assignable,trainable
id,,,
2,Autos & Vehicles,True,True
15,Pets & Animals,True,True
17,Sports,True,True
19,Travel & Events,True,True
20,Gaming,True,True
22,People & Blogs,True,True
23,Comedy,True,True
24,Entertainment,True,True
26,Howto & Style,True,True


In [16]:
def get_most_popular_for_category (category_id, max_results=5, page_token=None):
    request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        chart="mostPopular",
        maxResults=max_results,
        pageToken=page_token,
        regionCode="US",
        videoCategoryId=category_id
    )
    return request.execute()

In [17]:
most_viewed_sports = get_most_popular_for_category (17)
most_viewed_sports

{'kind': 'youtube#videoListResponse',
 'etag': 'CZkycxqjDRuJefh1Om4ktkGHBjA',
 'items': [{'kind': 'youtube#video',
   'etag': 'X6PzwIhg7gF305LbtY1rCa-4kgA',
   'id': 'owg5VD1Y4-E',
   'snippet': {'publishedAt': '2025-12-13T23:58:47Z',
    'channelId': 'UCVqiEQekNZlG6AjqyC_UJcg',
    'title': 'Ray Tells Jaylen Brown He’s Not Built Like IShowSpeed! 😭😂',
    'description': 'Ray Tells Jaylen Brown He’s Not Built Like IShowSpeed! 😭😂\n\n#ray #shorts #jaylenbrown #ishowspeed \n\nSubscribe For More!\n►https://www.youtube.com/channel/UCVqiEQekNZlG6AjqyC_Uscg',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/owg5VD1Y4-E/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/owg5VD1Y4-E/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/owg5VD1Y4-E/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/owg5VD1Y4-E/sddefault.jpg',
    

We've run into a bit of an issue here. seemingly all of the 